In [1]:
import sys
import os

# Get the absolute path of the directory where the notebook is currently running
# This usually is /path/to/app/folder_notebook
notebook_dir = os.getcwd()

# Get the absolute path of the 'app' directory (parent of folder_notebook)
main_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
if main_dir not in sys.path:
    sys.path.append(main_dir)

from utils import sql

In [2]:
db_dir = os.path.join(main_dir, 'data','planet_fone.db')
db_dir

'c:\\Users\\massi\\OneDrive\\Desktop\\Uni\\DS-M\\planetf1\\data\\planet_fone.db'

In [3]:
fg = sql.get_table('fone_geography',db_path=db_dir, verbose=True) 
tl = sql.get_table('travel_logistic',db_path=db_dir,verbose=True)

Connecting to database for table 'fone_geography' at: c:\Users\massi\OneDrive\Desktop\Uni\DS-M\planetf1\data\planet_fone.db
Fetching data from table 'fone_geography'...
Successfully fetched 48 rows from table 'fone_geography'.
Database connection closed.
Connecting to database for table 'travel_logistic' at: c:\Users\massi\OneDrive\Desktop\Uni\DS-M\planetf1\data\planet_fone.db
Fetching data from table 'travel_logistic'...
Successfully fetched 2256 rows from table 'travel_logistic'.
Database connection closed.


In [4]:
mad = [
    ('IFEMA', 'Madrid', 'Spain', 'Europe', 40.4636, -3.6162, 0)
]


In [5]:
import pandas as pd

mad_df = pd.DataFrame([mad[0]], columns=fg.columns[1:8])
for col in fg.columns:
    if col not in mad_df.columns:
        mad_df[col] = pd.NA
mad_df = mad_df[fg.columns]
mad_df

,id,circuit_x,city_x,country_x,continent,latitude,longitude,existing,months_to_avoid,traditional_months,notes,code_6,first_gp_probability,last_gp_probability
0,<NA>,IFEMA,Madrid,Spain,Europe,40.4636,-3.6162,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [6]:
mad_df['id'] = fg['id'].max() + 1
mad_df

,id,circuit_x,city_x,country_x,continent,latitude,longitude,existing,months_to_avoid,traditional_months,notes,code_6,first_gp_probability,last_gp_probability
0,49,IFEMA,Madrid,Spain,Europe,40.4636,-3.6162,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [7]:
mad_df['months_to_avoid'] = [[6, 7, 8]]
mad_df

,id,circuit_x,city_x,country_x,continent,latitude,longitude,existing,months_to_avoid,traditional_months,notes,code_6,first_gp_probability,last_gp_probability
0,49,IFEMA,Madrid,Spain,Europe,40.4636,-3.6162,0,"[6, 7, 8]",<NA>,<NA>,<NA>,<NA>,<NA>


In [9]:
mad_df['traditional_months'] = [[9]]

In [10]:
mad_df['notes'] = ['Madrid is very hot in summer.']
mad_df['code_6'] = ['SPAMAD']
mad_df['first_gp_probability'] = [0]
mad_df['last_gp_probability'] = [0]

In [11]:
mad_df

,id,circuit_x,city_x,country_x,continent,latitude,longitude,existing,months_to_avoid,traditional_months,notes,code_6,first_gp_probability,last_gp_probability
0,49,IFEMA,Madrid,Spain,Europe,40.4636,-3.6162,0,"[6, 7, 8]",[9],Madrid is very hot in summer.,SPAMAD,0,0


In [12]:
mad_log_df = pd.DataFrame(columns=tl.columns)

In [13]:
mad_log_df

,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes


In [20]:
new_rows = []
i = 1
for d in dest:
    new_rows.append({
        'id': tl['id'].max() + i,
        'from_id': mad_df['id'].values[0],
        'from_circuit': mad_df['circuit_x'].values[0],
        'to_id': d,
        'to_circuit': tl[tl['to_id'] == d]['to_circuit'].values[0],
    })
    i += 1
    new_rows.append({
        'id': tl['id'].max() + i,
        'from_id': d,
        'from_circuit': tl[tl['to_id'] == d]['to_circuit'].values[0],
        'to_id': mad_df['id'].values[0],
        'to_circuit': mad_df['circuit_x'].values[0]
    })
    i += 1

mad_log_df = pd.concat([mad_log_df, pd.DataFrame(new_rows)], ignore_index=True)


In [21]:
mad_log_df

,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes
0,2257,49,IFEMA,2,Budapest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2258,2,Budapest,49,IFEMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2259,49,IFEMA,3,Monza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2260,3,Monza,49,IFEMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2261,49,IFEMA,4,São Paulo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2348,47,Kuala Terengganu,49,IFEMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,2349,49,IFEMA,48,Vancouver,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,2350,48,Vancouver,49,IFEMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,2351,49,IFEMA,1,Silverstone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
import math

# Function to calculate distance using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [27]:
for from_id, from_circuit, to_id, to_circuit in zip(
        mad_log_df['from_id'], mad_log_df['from_circuit'], mad_log_df['to_id'], mad_log_df['to_circuit']
):
    if from_id==49:
        from_lat = mad_df[mad_df['id'] == from_id]['latitude'].values[0]
        from_lon = mad_df[mad_df['id'] == from_id]['longitude'].values[0]
    else:
        from_lat = fg[fg['id'] == from_id]['latitude'].values[0]
        from_lon = fg[fg['id'] == from_id]['longitude'].values[0]
    if to_id==49:
        to_lat = mad_df[mad_df['id'] == to_id]['latitude'].values[0]
        to_lon = mad_df[mad_df['id'] == to_id]['longitude'].values[0]
    else:
        to_lat = fg[fg['id'] == to_id]['latitude'].values[0]
        to_lon = fg[fg['id'] == to_id]['longitude'].values[0]
    distance = haversine(from_lat, from_lon, to_lat, to_lon)
    mad_log_df.loc[
        (mad_log_df['from_id'] == from_id) & (mad_log_df['to_id'] == to_id), 'distance_km'
    ] = distance
mad_log_df

,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes
0,2257,49,IFEMA,2,Budapest,1982.555592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2258,2,Budapest,49,IFEMA,1982.555592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2259,49,IFEMA,3,Monza,1192.477204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2260,3,Monza,49,IFEMA,1192.477204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2261,49,IFEMA,4,São Paulo,8411.495841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2348,47,Kuala Terengganu,49,IFEMA,11019.259505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,2349,49,IFEMA,48,Vancouver,8414.492638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,2350,48,Vancouver,49,IFEMA,8414.492638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,2351,49,IFEMA,1,Silverstone,1306.665029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
def needs_air_transport(from_country, to_country, from_continent, to_continent):
    island_countries = [
        'United Kingdom', 'Japan', 'Australia', 'Singapore', 'New Zealand',
        'Malaysia', 'Indonesia'
    ]
    isolated_countries = [
        'South Africa', 'Nigeria', 'South Korea'
    ]
    if from_continent != to_continent:
        return True
    if from_country in island_countries or to_country in island_countries:
        return True
    if from_country in isolated_countries or to_country in isolated_countries:
        return True
    if 'Oceania' in (from_continent, to_continent):
        return True
    return False
for from_id, to_id in zip(
        mad_log_df['from_id'], mad_log_df['to_id']
):
    if from_id==49:
        from_country = mad_df[mad_df['id'] == from_id]['country_x'].values[0]
        from_continent = mad_df[mad_df['id'] == from_id]['continent'].values[0]
    else:
        from_country = fg[fg['id'] == from_id]['country_x'].values[0]
        from_continent = fg[fg['id'] == from_id]['continent'].values[0]
    if to_id==49:
        to_country = mad_df[mad_df['id'] == to_id]['country_x'].values[0]
        to_continent = mad_df[mad_df['id'] == to_id]['continent'].values[0]
    else:
        to_country = fg[fg['id'] == to_id]['country_x'].values[0]
        to_continent = fg[fg['id'] == to_id]['continent'].values[0]
    air_transport = needs_air_transport(from_country, to_country, from_continent, to_continent)
    mad_log_df.loc[
        (mad_log_df['from_id'] == from_id) & (mad_log_df['to_id'] == to_id), 'needs_air'
    ] = 1 if air_transport else 0
mad_log_df

,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes
0,2257,49,IFEMA,2,Budapest,1982.555592,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2258,2,Budapest,49,IFEMA,1982.555592,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2259,49,IFEMA,3,Monza,1192.477204,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2260,3,Monza,49,IFEMA,1192.477204,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2261,49,IFEMA,4,São Paulo,8411.495841,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2348,47,Kuala Terengganu,49,IFEMA,11019.259505,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
92,2349,49,IFEMA,48,Vancouver,8414.492638,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
93,2350,48,Vancouver,49,IFEMA,8414.492638,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
94,2351,49,IFEMA,1,Silverstone,1306.665029,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
#using Openroute Service API to calculate distances for trucks
import openrouteservice
from openrouteservice.exceptions import ApiError
import time

def get_truck_route_info(origin, destination):
    """
    Given two (lat, lon) tuples, return distance in km and truck accessibility.
    
    Args:
        origin (tuple): (lat, lon) for the start point.
        destination (tuple): (lat, lon) for the end point.
        
    Returns:
        dict: {
            'distance_km': float or None,
            'truck_accessible': bool
        }
    """
    client = openrouteservice.Client(key='5b3ce3597851110001cf624867038ce71ba14d2c9cbdca5216627a22')
    
    try:
        print(f"Calculating route from {origin} to {destination}")
        route = client.directions(
            coordinates=[(origin[1], origin[0]), (destination[1], destination[0])],
            profile='driving-hgv',
            format='geojson'
        )
        
        distance_m = route['features'][0]['properties']['segments'][0]['distance']
        print(f"Distance: {distance_m} meters")
        return {
            'distance_km': round(distance_m / 1000, 2),
            'truck_accessible': True
        }
    
    except ApiError as e:
        # If the API can't find a route for trucks, it will raise an error
        print(f"Route error: {e}")
        return {
            'distance_km': None,
            'truck_accessible': False
        }

def rate_limited_get_truck_route_info(origin, destination, requests_per_minute=39):
    """
    Wrapper function to enforce rate limiting for API requests.
    
    Args:
        origin (tuple): (lat, lon) for the start point.
        destination (tuple): (lat, lon) for the end point.
        requests_per_minute (int): Maximum number of requests allowed per minute.
        
    Returns:
        dict: Result from get_truck_route_info.
    """
    delay = 60 / requests_per_minute  # Calculate delay between requests
    time.sleep(delay)  # Enforce delay
    return get_truck_route_info(origin, destination)

In [38]:
#testing api

magny_course = (46.8642, 3.1633)
monza = (45.6156, 9.2811)

result = get_truck_route_info(magny_course, monza)
print(result)

{'distance_km': 713.43, 'truck_accessible': True}


In [41]:
for i, row in mad_log_df.iterrows():
    if row['needs_air'] == 0:
        if row['from_id'] == 49:
            from_lat = mad_df[mad_df['id'] == row['from_id']]['latitude'].values[0]
            from_lon = mad_df[mad_df['id'] == row['from_id']]['longitude'].values[0]
        else:
            from_lat = fg[fg['id'] == row['from_id']]['latitude'].values[0]
            from_lon = fg[fg['id'] == row['from_id']]['longitude'].values[0]
        if row['to_id'] == 49:
            to_lat = mad_df[mad_df['id'] == row['to_id']]['latitude'].values[0]
            to_lon = mad_df[mad_df['id'] == row['to_id']]['longitude'].values[0]
        else:
            to_lat = fg[fg['id'] == row['to_id']]['latitude'].values[0]
            to_lon = fg[fg['id'] == row['to_id']]['longitude'].values[0]
        truck_route_info = rate_limited_get_truck_route_info((from_lat, from_lon), (to_lat, to_lon))
        mad_log_df.at[i, 'truck_distance_km'] = truck_route_info.get('distance_km')
        mad_log_df.at[i, 'truck_viable'] = truck_route_info.get('truck_accessible')

Calculating route from (40.4636, -3.6162) to (47.5789, 19.2486)
Distance: 2576941.9 meters
Calculating route from (47.5789, 19.2486) to (40.4636, -3.6162)
Distance: 2575972.0 meters
Calculating route from (40.4636, -3.6162) to (45.6156, 9.2811)
Distance: 1611155.8 meters
Calculating route from (45.6156, 9.2811) to (40.4636, -3.6162)
Distance: 1617026.8 meters
Calculating route from (40.4636, -3.6162) to (41.57, 2.2611)
Distance: 617581.2 meters
Calculating route from (41.57, 2.2611) to (40.4636, -3.6162)
Distance: 623020.3 meters
Calculating route from (40.4636, -3.6162) to (47.2196, 14.7649)
Distance: 2218698.3 meters
Calculating route from (47.2196, 14.7649) to (40.4636, -3.6162)
Distance: 2217128.9 meters
Calculating route from (40.4636, -3.6162) to (49.3249, 8.5694)
Distance: 1750690.7 meters
Calculating route from (49.3249, 8.5694) to (40.4636, -3.6162)
Distance: 1758547.6 meters
Calculating route from (40.4636, -3.6162) to (44.3439, 11.7167)
Distance: 1762547.7 meters
Calculating

In [45]:
import numpy as np

mad_log_df['truck_viable'] = np.where(
    mad_log_df['truck_viable']== True, 1,
    np.where(mad_log_df['truck_viable'] == False, 0, np.nan)
)

In [46]:
mad_log_df

,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes
0,2257,49,IFEMA,2,Budapest,1982.555592,NaN,0,2576.94,1.0,NaN,NaN,NaN,NaN
1,2258,2,Budapest,49,IFEMA,1982.555592,NaN,0,2575.97,1.0,NaN,NaN,NaN,NaN
2,2259,49,IFEMA,3,Monza,1192.477204,NaN,0,1611.16,1.0,NaN,NaN,NaN,NaN
3,2260,3,Monza,49,IFEMA,1192.477204,NaN,0,1617.03,1.0,NaN,NaN,NaN,NaN
4,2261,49,IFEMA,4,São Paulo,8411.495841,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2348,47,Kuala Terengganu,49,IFEMA,11019.259505,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
92,2349,49,IFEMA,48,Vancouver,8414.492638,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
93,2350,48,Vancouver,49,IFEMA,8414.492638,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
94,2351,49,IFEMA,1,Silverstone,1306.665029,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
import numpy as np

mad_log_df['needs_air'] = np.where(
    (mad_log_df['needs_air'] == 0) & (mad_log_df['truck_viable'] == 0),
    1,
    mad_log_df['needs_air']
)
mad_log_df

,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes
0,2257,49,IFEMA,2,Budapest,1982.555592,NaN,0,2576.94,1.0,NaN,NaN,NaN,NaN
1,2258,2,Budapest,49,IFEMA,1982.555592,NaN,0,2575.97,1.0,NaN,NaN,NaN,NaN
2,2259,49,IFEMA,3,Monza,1192.477204,NaN,0,1611.16,1.0,NaN,NaN,NaN,NaN
3,2260,3,Monza,49,IFEMA,1192.477204,NaN,0,1617.03,1.0,NaN,NaN,NaN,NaN
4,2261,49,IFEMA,4,São Paulo,8411.495841,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2348,47,Kuala Terengganu,49,IFEMA,11019.259505,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
92,2349,49,IFEMA,48,Vancouver,8414.492638,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
93,2350,48,Vancouver,49,IFEMA,8414.492638,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
94,2351,49,IFEMA,1,Silverstone,1306.665029,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# Emission Factor
# Air: ~420 g CO₂ / tonne-km
# Truck: ~42 g CO₂ / tonne-km

# Total CO₂ (kg) = Distance (km) × Cargo Mass (tonnes) × Emission Factor (g/tonne-km) ÷ 1000

# truck_emissions = truck_distance_km x 1000 x 420 / 1000
# air_emissions = distance_km x 1000 x 42 / 1000
mad_log_df['truck_emissions'] = mad_log_df['truck_distance_km'] * 1000 * 42 / 1000
mad_log_df['air_emissions'] = mad_log_df['distance_km'] * 1000 * 420 / 1000
mad_log_df


,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes
0,2257,49,IFEMA,2,Budapest,1982.555592,NaN,0,2576.94,1,108231.48,832673.348471,NaN,NaN
1,2258,2,Budapest,49,IFEMA,1982.555592,NaN,0,2575.97,1,108190.74,832673.348471,NaN,NaN
2,2259,49,IFEMA,3,Monza,1192.477204,NaN,0,1611.16,1,67668.72,500840.425861,NaN,NaN
3,2260,3,Monza,49,IFEMA,1192.477204,NaN,0,1617.03,1,67915.26,500840.425861,NaN,NaN
4,2261,49,IFEMA,4,São Paulo,8411.495841,NaN,1,NaN,0,NaN,3532828.253121,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2348,47,Kuala Terengganu,49,IFEMA,11019.259505,NaN,1,NaN,0,NaN,4628088.992209,NaN,NaN
92,2349,49,IFEMA,48,Vancouver,8414.492638,NaN,1,NaN,0,NaN,3534086.907945,NaN,NaN
93,2350,48,Vancouver,49,IFEMA,8414.492638,NaN,1,NaN,0,NaN,3534086.907945,NaN,NaN
94,2351,49,IFEMA,1,Silverstone,1306.665029,NaN,1,NaN,1,NaN,548799.31222,NaN,NaN


In [52]:
mad_log_df['truck_viable'] = mad_log_df['truck_viable'].astype('Int64')
mad_log_df['truck_viable'].fillna(0, inplace=True)
mad_log_df

,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes
0,2257,49,IFEMA,2,Budapest,1982.555592,NaN,0,2576.94,1,1082314.8,83267.334847,NaN,NaN
1,2258,2,Budapest,49,IFEMA,1982.555592,NaN,0,2575.97,1,1081907.4,83267.334847,NaN,NaN
2,2259,49,IFEMA,3,Monza,1192.477204,NaN,0,1611.16,1,676687.2,50084.042586,NaN,NaN
3,2260,3,Monza,49,IFEMA,1192.477204,NaN,0,1617.03,1,679152.6,50084.042586,NaN,NaN
4,2261,49,IFEMA,4,São Paulo,8411.495841,NaN,1,NaN,0,NaN,353282.825312,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2348,47,Kuala Terengganu,49,IFEMA,11019.259505,NaN,1,NaN,0,NaN,462808.899221,NaN,NaN
92,2349,49,IFEMA,48,Vancouver,8414.492638,NaN,1,NaN,0,NaN,353408.690795,NaN,NaN
93,2350,48,Vancouver,49,IFEMA,8414.492638,NaN,1,NaN,0,NaN,353408.690795,NaN,NaN
94,2351,49,IFEMA,1,Silverstone,1306.665029,NaN,1,NaN,0,NaN,54879.931222,NaN,NaN


In [54]:
# Set truck_viable = 0 for routes with truck_distance_km > 2200 and not both in same continent (Europe, North America, South America)
continents = ['Europe', 'North America', 'South America']

def get_continent(id_):
    if id_ == 49:
        # Madrid (IFEMA) is in Europe
        return 'Europe'
    return fg.loc[fg['id'] == int(id_), 'continent'].values[0]

mask = (
    mad_log_df['truck_distance_km'].notnull() &
    ~(
        mad_log_df.apply(lambda row: get_continent(row['from_id']) == get_continent(row['to_id']) and get_continent(row['from_id']) in continents, axis=1)
    ) &
    (mad_log_df['truck_distance_km'].astype(float) > 2200)
)

mad_log_df.loc[mask, 'truck_viable'] = 0
mad_log_df

,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes
0,2257,49,IFEMA,2,Budapest,1982.555592,NaN,0,2576.94,1,1082314.8,83267.334847,NaN,NaN
1,2258,2,Budapest,49,IFEMA,1982.555592,NaN,0,2575.97,1,1081907.4,83267.334847,NaN,NaN
2,2259,49,IFEMA,3,Monza,1192.477204,NaN,0,1611.16,1,676687.2,50084.042586,NaN,NaN
3,2260,3,Monza,49,IFEMA,1192.477204,NaN,0,1617.03,1,679152.6,50084.042586,NaN,NaN
4,2261,49,IFEMA,4,São Paulo,8411.495841,NaN,1,NaN,0,NaN,353282.825312,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2348,47,Kuala Terengganu,49,IFEMA,11019.259505,NaN,1,NaN,0,NaN,462808.899221,NaN,NaN
92,2349,49,IFEMA,48,Vancouver,8414.492638,NaN,1,NaN,0,NaN,353408.690795,NaN,NaN
93,2350,48,Vancouver,49,IFEMA,8414.492638,NaN,1,NaN,0,NaN,353408.690795,NaN,NaN
94,2351,49,IFEMA,1,Silverstone,1306.665029,NaN,1,NaN,0,NaN,54879.931222,NaN,NaN


In [59]:
mad_log_df.loc[
    (mad_log_df['from_circuit'] == 'Silverstone') | (mad_log_df['to_circuit'] == 'Silverstone'),
    'truck_viable'
] = 1

mad_log_df.loc[
    (mad_log_df['from_circuit'] == 'Silverstone') | (mad_log_df['to_circuit'] == 'Silverstone'),
    'truck_distance_km'
] = 1850

mad_log_df.loc[
    (mad_log_df['from_circuit'] == 'Silverstone') | (mad_log_df['to_circuit'] == 'Silverstone'),
    'truck_emissions'
] = 1850 * 42

mad_log_df

,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes
0,2257,49,IFEMA,2,Budapest,1982.555592,NaN,0,2576.94,1,108231.48,832673.348471,231.924600,NaN
1,2258,2,Budapest,49,IFEMA,1982.555592,NaN,0,2575.97,1,108190.74,832673.348471,231.837300,NaN
2,2259,49,IFEMA,3,Monza,1192.477204,NaN,0,1611.16,1,67668.72,500840.425861,145.004400,NaN
3,2260,3,Monza,49,IFEMA,1192.477204,NaN,0,1617.03,1,67915.26,500840.425861,145.532700,NaN
4,2261,49,IFEMA,4,São Paulo,8411.495841,NaN,1,NaN,0,NaN,3532828.253121,8411.495841,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2348,47,Kuala Terengganu,49,IFEMA,11019.259505,NaN,1,NaN,0,NaN,4628088.992209,11019.259505,NaN
92,2349,49,IFEMA,48,Vancouver,8414.492638,NaN,1,NaN,0,NaN,3534086.907945,8414.492638,NaN
93,2350,48,Vancouver,49,IFEMA,8414.492638,NaN,1,NaN,0,NaN,3534086.907945,8414.492638,NaN
94,2351,49,IFEMA,1,Silverstone,1306.665029,NaN,1,1850,1,77700,548799.31222,1306.665029,NaN


In [60]:
mad_log_df['effort_score'] = mad_log_df.apply(
    lambda row: float(row['truck_distance_km']) * 0.09 if row['truck_viable'] == 1 and pd.notnull(row['truck_distance_km'])
    else float(row['distance_km']) if pd.notnull(row['distance_km'])
    else pd.NA,
    axis=1
)
mad_log_df

,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes
0,2257,49,IFEMA,2,Budapest,1982.555592,NaN,0,2576.94,1,108231.48,832673.348471,231.924600,NaN
1,2258,2,Budapest,49,IFEMA,1982.555592,NaN,0,2575.97,1,108190.74,832673.348471,231.837300,NaN
2,2259,49,IFEMA,3,Monza,1192.477204,NaN,0,1611.16,1,67668.72,500840.425861,145.004400,NaN
3,2260,3,Monza,49,IFEMA,1192.477204,NaN,0,1617.03,1,67915.26,500840.425861,145.532700,NaN
4,2261,49,IFEMA,4,São Paulo,8411.495841,NaN,1,NaN,0,NaN,3532828.253121,8411.495841,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2348,47,Kuala Terengganu,49,IFEMA,11019.259505,NaN,1,NaN,0,NaN,4628088.992209,11019.259505,NaN
92,2349,49,IFEMA,48,Vancouver,8414.492638,NaN,1,NaN,0,NaN,3534086.907945,8414.492638,NaN
93,2350,48,Vancouver,49,IFEMA,8414.492638,NaN,1,NaN,0,NaN,3534086.907945,8414.492638,NaN
94,2351,49,IFEMA,1,Silverstone,1306.665029,NaN,1,1850,1,77700,548799.31222,166.500000,NaN


In [61]:
mad_log_df['codes'] = mad_log_df.apply(
    lambda row: f"{mad_df.loc[mad_df['id'] == row['from_id'], 'code_6'].values[0]}-{fg.loc[fg['id'] == int(row['to_id']), 'code_6'].values[0]}"
    if int(row['from_id']) == 49
    else f"{fg.loc[fg['id'] == int(row['from_id']), 'code_6'].values[0]}-{mad_df.loc[mad_df['id'] == row['to_id'], 'code_6'].values[0]}",
    axis=1
)
mad_log_df

,id,from_id,from_circuit,to_id,to_circuit,distance_km,transport_mode,needs_air,truck_distance_km,truck_viable,truck_emissions,air_emissions,effort_score,codes
0,2257,49,IFEMA,2,Budapest,1982.555592,NaN,0,2576.94,1,108231.48,832673.348471,231.924600,SPAMAD-HUNBUD
1,2258,2,Budapest,49,IFEMA,1982.555592,NaN,0,2575.97,1,108190.74,832673.348471,231.837300,HUNBUD-SPAMAD
2,2259,49,IFEMA,3,Monza,1192.477204,NaN,0,1611.16,1,67668.72,500840.425861,145.004400,SPAMAD-ITAMON
3,2260,3,Monza,49,IFEMA,1192.477204,NaN,0,1617.03,1,67915.26,500840.425861,145.532700,ITAMON-SPAMAD
4,2261,49,IFEMA,4,São Paulo,8411.495841,NaN,1,NaN,0,NaN,3532828.253121,8411.495841,SPAMAD-BRASAO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2348,47,Kuala Terengganu,49,IFEMA,11019.259505,NaN,1,NaN,0,NaN,4628088.992209,11019.259505,MALKUA-SPAMAD
92,2349,49,IFEMA,48,Vancouver,8414.492638,NaN,1,NaN,0,NaN,3534086.907945,8414.492638,SPAMAD-CANVAN
93,2350,48,Vancouver,49,IFEMA,8414.492638,NaN,1,NaN,0,NaN,3534086.907945,8414.492638,CANVAN-SPAMAD
94,2351,49,IFEMA,1,Silverstone,1306.665029,NaN,1,1850,1,77700,548799.31222,166.500000,SPAMAD-UKGSIL


In [62]:
db_dir

'c:\\Users\\massi\\OneDrive\\Desktop\\Uni\\DS-M\\planetf1\\data\\planet_fone.db'

In [71]:
# Prepare the row for insertion
row = mad_df.iloc[0].copy()
row['id'] = int(row['id'])
row['existing'] = int(row['existing'])
row['first_gp_probability'] = float(row['first_gp_probability'])
row['last_gp_probability'] = float(row['last_gp_probability'])
row['months_to_avoid'] = str(row['months_to_avoid'])
row['traditional_months'] = str(row['traditional_months'])

columns = ', '.join(mad_df.columns)
placeholders = ', '.join(['?'] * len(mad_df.columns))
values = [row[col] for col in mad_df.columns]

conn = sqlite3.connect(db_dir)
cursor = conn.cursor()
cursor.execute(f"INSERT INTO fone_geography ({columns}) VALUES ({placeholders})", values)
conn.commit()
conn.close()


In [73]:
mad_log_df.dtypes

id                    object
from_id               object
from_circuit          object
to_id                 object
to_circuit            object
distance_km           object
transport_mode        object
needs_air             object
truck_distance_km     object
truck_viable           Int64
truck_emissions       object
air_emissions         object
effort_score         float64
codes                 object
dtype: object

In [74]:
import sqlite3

# Prepare mad_log_df for insertion
mad_log_df_insert = mad_log_df.copy()

# Ensure correct dtypes
mad_log_df_insert['id'] = mad_log_df_insert['id'].astype(int)
mad_log_df_insert['from_id'] = mad_log_df_insert['from_id'].astype(int)
mad_log_df_insert['to_id'] = mad_log_df_insert['to_id'].astype(int)
mad_log_df_insert['needs_air'] = mad_log_df_insert['needs_air'].astype(int)
mad_log_df_insert['truck_viable'] = mad_log_df_insert['truck_viable'].astype(int)
mad_log_df_insert['distance_km'] = mad_log_df_insert['distance_km'].astype(float)
mad_log_df_insert['truck_distance_km'] = mad_log_df_insert['truck_distance_km'].astype(float)
mad_log_df_insert['truck_emissions'] = mad_log_df_insert['truck_emissions'].astype(float)
mad_log_df_insert['air_emissions'] = mad_log_df_insert['air_emissions'].astype(float)
mad_log_df_insert['effort_score'] = mad_log_df_insert['effort_score'].astype(float)

# Only keep columns that exist in the travel_logistic table
cols_to_insert = tl.columns
mad_log_df_insert = mad_log_df_insert[cols_to_insert]

# Insert into SQLite

with sqlite3.connect(db_dir) as conn:
    mad_log_df_insert.to_sql('travel_logistic', conn, if_exists='append', index=False)